## Import Libraries

Mix of libraries for data preprocessing, visualization, and modeling.

In [1]:
import random
random.seed(42)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import time

from PIL import Image
import imagesize

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import layers, models

from scipy.ndimage import sobel
from keras import callbacks

## Exploration

Here, we will be creating a dataframe with all our image paths (as strings), bloodcell type, and image dimension information. We will take a look at if our data is balanced or not with the bloodcell type counts and the image dimension counts.

In [2]:
# Dataset folder has 8 different folders, which represent 8 different bloodcells we will be classifying
# Open up folder names and remove folder names that are not bloodcell types

# Original kaggle dataset has images stored in each of the 8 folders, so we made a folder that contained all the images
# so that it is easier to convert the images to numpy arrays later on

bloodcells = os.listdir("bloodcells_dataset")
bloodcells = [x for x in bloodcells if x not in ['.DS_Store', 'All_Images']]

bloodcells

['basophil',
 'neutrophil',
 'ig',
 'monocyte',
 'eosinophil',
 'erythroblast',
 'lymphocyte',
 'platelet']

In [3]:
def image_df(folder_names):

    '''
    Outputs a dataframe for image paths (as strings), bloodcell type, and image dimension information.

    Args:
        1) folder_names (list): list of bloodcell type folders
    '''

    # initialize empty list to store dataframes that contain image strings and bloodcell type
    dfs = []

    # loop through bloodcell types and store image paths and bloodcell categories
    for i in range(len(folder_names)):
        
        images = os.listdir('bloodcells_dataset/' + folder_names[i]) # jpg string paths
        
        df = pd.DataFrame(data = {'images': images, 'type': folder_names[i]})
        
        dfs.append(df)

    all_data = pd.concat(dfs)

    all_data = all_data[all_data['images'].str.contains('.DS_Store') == False]

    # Convert bloodcell types to numbers for our model
    le = LabelEncoder()
    all_data['type'] = le.fit_transform(all_data['type'])

    # Store dimensions of image incase we find different dimensions 
    dimensions = pd.Series([imagesize.get('bloodcells_dataset/All_Images/' + x) for x in all_data['images']])
    widths, heights = map(list, zip(*dimensions))
    all_data['width'] = widths
    all_data['height'] = heights

    # Reset index and remove image paths that may have been accidentally copied
    all_data = all_data[all_data['images'].str.contains('copy') == False]
    all_data = all_data.reset_index(drop = True)
    
    return all_data


    

In [4]:
df = image_df(bloodcells)

df

,images,type,width,height
0,BA_689200.jpg,0,360,363
1,BA_883452.jpg,0,360,363
2,BA_382161.jpg,0,366,369
3,BA_175579.jpg,0,360,363
4,BA_775722.jpg,0,360,363
...,...,...,...,...
17087,PLATELET_495918.jpg,7,360,363
17088,PLATELET_897238.jpg,7,360,363
17089,PLATELET_750430.jpg,7,360,363
17090,PLATELET_810431.jpg,7,360,363


In [5]:
# Visuals for count of height/width and count of each blood cell type
# Maybe use function to plot

# df[['height', 'width']].value_counts().reset_index(name = 'count') 
# df[['type']].value_counts().reset_index(name = 'count')

In [6]:
class Sampling:

    def __init__(self, data, sampling_method):

        self.data = data
        
        self.sampling_method = sampling_method

    def sample_data(self, sampling_percent = 0.8):

        df = self.data.copy()

        category_counts = df[['type']].value_counts().reset_index(name = 'count')

        train_dfs = []

        if self.sampling_method == 'weighted':

            for i in range(len(category_counts)):

                type = category_counts['type'][i]
            
                if category_counts['count'][i] >= 2000:
                    add_samples = df[df['type'] == type].sample(1500)
                else: 
                    add_samples = df[df['type'] == type].sample(1000)

                train_dfs.append(add_samples)
            
            train = pd.concat(train_dfs)

            remaining_data = df[~df['images'].isin(train['images'])]
    
            validation = remaining_data.sample(int(len(remaining_data) / 2))
        
        elif self.sampling_method == 'proportional': 

            num_samples = int(sampling_percent * len(df))
            
            category_counts['prop'] = category_counts['count'] / len(df)
            category_counts['train_samples'] = (category_counts['prop'] * num_samples).astype('int32')
            category_counts['val_samples'] = (category_counts['train_samples'] * 0.2).astype('int32')
            category_counts['train_samples'] = category_counts['train_samples'] - category_counts['val_samples']

            val_dfs = []

            for i in range(len(category_counts)):

                type = category_counts['type'][i]
            
                samples = category_counts['train_samples'][i]
            
                add_samples = df[df['type'] == type].sample(samples)

                train_dfs.append(add_samples)
            
            train = pd.concat(train_dfs)

            remaining_data = df[~df['images'].isin(train['images'])]

            for i in range(len(category_counts)):

                type = category_counts['type'][i]
            
                samples = category_counts['val_samples'][i]
            
                add_samples = remaining_data[remaining_data['type'] == type].sample(samples)

                val_dfs.append(add_samples)
            
            validation = pd.concat(val_dfs)

        train = train.dropna(how = 'all')

        float_cols = train.select_dtypes(np.number)

        train[float_cols.columns] = float_cols.astype('int32')

        test = remaining_data[~remaining_data['images'].isin(validation['images'])]

        return train, validation, test
            

In [7]:
weighted_sampling = Sampling(df, 'weighted')
proportional_sampling = Sampling(df, 'proportional')

weighted_train, weighted_val, weighted_test = weighted_sampling.sample_data()
prop_train, prop_val, prop_test = proportional_sampling.sample_data()

In [8]:
class Convert_Images:

    def __init__(self, data):

        self.data = data

        self.file_names = (self.data)['images'].apply(lambda x: 'bloodcells_dataset/All_Images/' + x)

        self.labels = self.data['type']

    def load_image(self, file_name, resize, edge):
        '''
        Args:
            file_name:
            resize: 
            edge (boolean): if True, function will operate to prepare edge detected data. If false, function will normalize accordingly
        '''
        
        raw = tf.io.read_file(file_name)
        
        tensor = tf.io.decode_image(raw, expand_animations = False)
        
        tensor = tf.image.resize(tensor, size = [resize, resize])

        if edge == True:
            tensor = tf.cast(tensor, tf.float32)
        else:
            tensor = tf.cast(tensor, tf.float32) / 255.0
        
        return tensor

    def image_arrays_and_labels(self, edge, resize = 32):

      dataset = tf.data.Dataset.from_tensor_slices(self.file_names)
        
      dataset = dataset.map(lambda file_name: self.load_image(file_name, resize, edge))
        
      images = np.array(list(dataset))
        
      return images, self.labels

In [9]:
weighted_train_image_df, weighted_val_image_df, weighted_test_image_df = Convert_Images(weighted_train), Convert_Images(weighted_val), Convert_Images(weighted_test) 
prop_train_image_df, prop_val_image_df, prop_test_image_df = Convert_Images(prop_train), Convert_Images(prop_val), Convert_Images(prop_test)



resize_pixels = 32

weighted_train_images, weighted_train_labels = weighted_train_image_df.image_arrays_and_labels(resize_pixels)
weighted_val_images, weighted_val_labels = weighted_val_image_df.image_arrays_and_labels(resize_pixels)
weighted_test_images, weighted_test_labels = weighted_test_image_df.image_arrays_and_labels(resize_pixels)

prop_train_images, prop_train_labels = prop_train_image_df.image_arrays_and_labels(resize_pixels)
prop_val_images, prop_val_labels = prop_val_image_df.image_arrays_and_labels(resize_pixels)
prop_test_images, prop_test_labels = prop_test_image_df.image_arrays_and_labels(resize_pixels)

2024-06-10 19:27:29.679264: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 19:27:30.302858: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 19:27:30.740478: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 19:27:31.868885: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 19:27:32.258881: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 19:27:32.658955: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
def edge_detection(input_data):
    '''
    Args:
        input_data (array): image array
    Returns:
        x_sobel (array): horizontal sobel-filtered array with same dimension as input_data
        y_sobel (array): vertical sobel-filtered array with same dimension as input_data
        norm (array): combined array magnitude after filter application
    '''
    input_data = input_data.astype('int32')
    x_sobel = sobel(input_data, axis = 0) #compute horizontal gradient
    y_sobel = sobel(input_data, axis = 1) #compute vertical gradient
    norm = np.sqrt(x_sobel**2 + y_sobel**2) #compute norm
    norm *= 255.0 / np.max(norm)  # normalization
    return x_sobel, y_sobel, norm

In [23]:
edge_weighted_train, edge_weighted_val, edge_weighted_test = Convert_Images(weighted_train), Convert_Images(weighted_val), Convert_Images(weighted_test)

w_edge_train_images, w_edge_train_labels = edge_weighted_train.image_arrays_and_labels(True, resize_pixels)
w_edge_val_images, w_edge_val_labels = edge_weighted_val.image_arrays_and_labels(True, resize_pixels)
w_edge_test_images, w_edge_test_labels = edge_weighted_test.image_arrays_and_labels(True, resize_pixels)

2024-06-10 19:30:41.312729: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 19:30:41.756995: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-10 19:30:42.170796: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [26]:
#apply sobel filter vertically and horizontally for edge detection
x_sobel, y_sobel, norm = edge_detection(w_edge_train_images)
x_sobel_val, y_sobel_val, norm_val = edge_detection(w_edge_val_images)
x_sobel_test, y_sobel_test, norm_test = edge_detection(w_edge_test_images)

In [13]:
def plot_edges(image, x_sobel, y_sobel, norm):
    '''
    Plot images after edge detection.
    
    Args:
        input_data
        x_sobel
        y_sobel
        norm
    
    Returns 4 subplots revealing horizontal + vertical edge detection before and after.
    '''
    image = image.astype('int32')
    x_sobel = x_sobel.astype("int32")
    y_sobel = y_sobel.astype("int32")
    norm = norm.astype("int32")
    fig, ax = plt.subplots(2, 2, figsize=(100, 100))
    #plt.gray()  # show the filtered result in grayscale
    ax[0, 0].imshow(image[1])
    ax[0, 1].imshow(x_sobel[1])
    ax[1, 0].imshow(y_sobel[1])
    ax[1, 1].imshow(norm[1])
    #plt.gray()  # show the filtered result in grayscale
    titles = ["original", "horizontal", "vertical", "norm"]
    for i, ax in enumerate(ax.ravel()):
        ax.set_title(titles[i])
        ax.axis("off")
    plt.show()

In [27]:
plot_edges(w_edge_val_images, x_sobel_val, y_sobel_val, norm_val)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [15]:
print(f'weighted_train_images: {len(weighted_train_images)}')
print(f'weighted_val_images: {len(weighted_val_images)}')
print(f'weighted_test_images: {len(weighted_test_images)}')
print(f'prop_train_images: {len(prop_train_images)}')
print(f'prop_val_images: {len(prop_val_images)}')
print(f'prop_test_images: {len(prop_test_images)}')

weighted_train_images: 10000
weighted_val_images: 3546
weighted_test_images: 3546
prop_train_images: 10938
prop_val_images: 2731
prop_test_images: 3423


In [30]:
model2 = models.Sequential(
    
    [
        # 32 kernels, 3 by 3 kernel
        # input is 300 by 300 pixels with 3 color channels
        layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (resize_pixels, resize_pixels, 3)),
        #layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.Dropout(0.2),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3,3), activation = "relu"),
        layers.Dropout(0.2),
        layers.Conv2D(64, (3, 3), activation = 'relu'),
        #layers.Conv2D(64, (3,3), activation = "relu"),
        #layers.MaxPooling2D((2, 2)),
        #layers.Conv2D(64, (3,3), activation = "relu"),
        #layers.Conv2D(64, (3,3), activation = "relu"),
        # flatten into 1d array
        layers.Flatten(),

        # Neural network
        layers.Dense(64, activation = 'relu'),

        # 10 different categories
        layers.Dense(8) 
    ]
    
)

model2.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), # using from_logits = True b/c no softmax layer
              metrics = ['accuracy'])

earlystopping = callbacks.EarlyStopping(monitor="val_accuracy",
                                        mode="min",
                                        patience=5,
                                        restore_best_weights=True)


history = model2.fit(norm, 
                    w_edge_train_labels, 
                    epochs = 3, 
                    batch_size = 32,
                    validation_data=(norm_val, w_edge_val_labels),
                    callbacks=[earlystopping]
                   )

Epoch 1/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.5521 - loss: 1.2952 - val_accuracy: 0.2899 - val_loss: 2.6084
Epoch 2/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.8332 - loss: 0.4564 - val_accuracy: 0.2913 - val_loss: 3.3356
Epoch 3/3
313/313 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.8806 - loss: 0.3288 - val_accuracy: 0.2908 - val_loss: 2.9259


In [32]:
predictions = model2.predict(w_edge_val_labels)

predictions = predictions.argmax(axis = 1)

np.array(predictions == w_edge_val_labels).sum() / len(predictions)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_5_1/Cast:0", shape=(32, 1), dtype=float32). Expected shape (None, 32, 32, 3), but input has incompatible shape (32, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 1), dtype=int64)
  • training=False
  • mask=None